## Exercise 2: Building a simple agentic personalized customer emailing system

In [ ]:
# set your open ai key in .env file (text file) as OPENAI_API_KEY=your-key
# your .env file should be in tthe same directory as this file
from dotenv import load_dotenv
_ = load_dotenv()

In [ ]:
# agentic_email_demo.py
#
# Setup:
#   pip install -r requirements.txt
#   set OPENAI_API_KEY in your environment.

from typing import Dict, Any, List

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.agents import create_agent

# ---------------------------
# Mock tools (Act)
# ---------------------------

CUSTOMER_DB: Dict[str, Dict[str, Any]] = {
    "123": {
        "name": "Alex Johnson",
        "email": "alex.johnson@example.com",
        "purchases": [
            {"item": "Premium Plan", "date": "2024-11-02", "amount": 99},
            {"item": "Addon Storage", "date": "2025-01-10", "amount": 19},
        ],
        "issues": [
            {"date": "2025-01-15", "summary": "Billing confusion on renewal"},
        ],
    }
}


@tool
def get_customer_history(customer_id: str) -> str:
    """Look up a customer's past purchases and support issues (mock)."""
    data = CUSTOMER_DB.get(customer_id)
    if not data:
        return f"No data found for customer_id={customer_id}"

    lines: List[str] = []
    lines.append(f"Name: {data['name']}")
    lines.append(f"Email: {data['email']}")
    lines.append("Recent purchases:")
    for p in data["purchases"]:
        lines.append(f"- {p['date']}: {p['item']} (${p['amount']})")
    lines.append("Recent support issues:")
    for i in data["issues"]:
        lines.append(f"- {i['date']}: {i['summary']}")
    return "\n".join(lines)


SENT_EMAIL_LOG: List[Dict[str, str]] = []  # tiny "memory" for learning


@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an email to the customer (mock)."""
    email = {"to": to, "subject": subject, "body": body}
    SENT_EMAIL_LOG.append(email)
    # Real code would call an email API here.
    return f"EMAIL_SENT to={to} subject={subject}"


# ---------------------------
# Model + Agent
# ---------------------------

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

SYSTEM_PROMPT = """
You are a customer-success agent for a subscription SaaS product.

Tools available:
- get_customer_history(customer_id: str): returns a text summary of the
  customer's purchases and issues.
- send_email(to: str, subject: str, body: str): sends an email (mock).

Perceive:
- Understand the user's request and identify the customer ID if present.

Decide:
- If the request references a specific customer, use get_customer_history
  before writing the email.

Act:
- Draft a personalized, empathetic email and send it with send_email.

Learn:
- After sending the email, briefly explain why this message is better than
  a generic template, so the team can improve future outreach.
"""

tools = [get_customer_history, send_email]

# NOTE: this is the current signature: model=..., tools=..., system_prompt=...
agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=SYSTEM_PROMPT,
)

# ---------------------------
# Run the agent
# ---------------------------

if __name__ == "__main__":
    user_request = (
        "Customer 123 was upset about a billing issue last month. "
        "Write them a personalized apology email that references their "
        "recent purchases and offers a 20% discount on their next renewal. "
        "Please go ahead and send the email."
    )

    # For create_agent, you send messages as a list of role/content dicts.
    result = agent.invoke(
        {
            "messages": [
                {"role": "user", "content": user_request},
            ]
        }
    )

    print("\n=== Final agent reply ===")
    # Final reply is the last message in the messages list
    print(result["messages"][-1].content)

    print("\n=== Sent email log (mock memory) ===")
    for email in SENT_EMAIL_LOG:
        print("-" * 40)
        print("To:", email["to"])
        print("Subject:", email["subject"])
        print("Body:\n", email["body"])